In [75]:
from sklearn import decomposition
import random  
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
from scipy import signal

# Import the mat data return the mat_header, all the experiment_data, recording time vector

In [76]:
def convert_mat(mat_name):
    raw_channel_recording = sp.io.loadmat(mat_name,mat_dtype = True)
    index_time = raw_channel_recording.get('t_s') 
    channel_recording = raw_channel_recording.get('spikes')
    header = raw_channel_recording.get('__header__')
    channel_recording = channel_recording[4:,:]# it could be really flexible
    return header, channel_recording,index_time

# convert the channel recording for each experiment into a dataframe
## return the experiment_collection(a dict with keys named by the order of experiment
## return the new recording length for this time_window

In [84]:
def to_timeseries(experiment_selected,time_resolution,index_time):
    channel_collection= []
    for i in np.arange(0,64):
        channel_collection.append('Channel_{0}'.format(i))
    order = 0
    steps = time_resolution/0.00008 # 0.00008 is the highest time resolution in the raw data # turn the time resolution into steps
    ##Binary conversion
    experiment_collection ={}
    for experiment in experiment_selected:
        temp_df = pd.DataFrame(np.zeros([np.ceil(len(index_time/steps),64)]),columns = channel_collection)
        for i,channel in enumerate(experiment): 
            if len(channel[0]) != 0:
                channel = np.floor(channel/steps)
                np.reshape(channel,len(channel))
                channel = channel.astype(int)
                channel = np.reshape(channel,len(channel))
                temp_df.loc[channel,'Channel_{0}'.format(i)]+= 1
        temp_df.set_index(np.arange(0,np.max(index_time),time_resolution))
        experiment_collection['experiment_{0}'.format(order+5)]=temp_df
        order+=1
    recording_length = len(temp_df.index)
    return experiment_collection,recording_length


SyntaxError: invalid syntax (<ipython-input-84-a190c2bd27d3>, line 10)

# Find the time windows that in each window there is only one spike(or signal)
## return a number

In [78]:
def highest_timeresolution_withoneineach(experiment_selected):
    #find the minimum time window  
    list_min=[]
    for experiment in experiment_selected:
        for i,channel in enumerate(experiment):
            if len(channel[0]) != 0:
                np.reshape(channel,len(channel))
                channel = channel.astype(int)
                channel = np.reshape(channel,len(channel))
                channel =np.append(channel,0)
                list_min.append(np.min(np.abs(channel[0:-1]-channel[1:])))
    return np.min(list_min)

# Convolve https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve.html
# decomposition http://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition

In [80]:
header,channel_recording,order_time = convert_mat('forstan.mat')

In [81]:
experiment_selected,recording_length = to_timeseries(channel_recording[1:3],0.1,order_time)


D:\new\anaconda\lib\site-packages\ipykernel\__main__.py:10: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [74]:
experiment_selected.keys()

dict_keys(['experiment_5', 'experiment_6'])